In [ ]:
from IPython.display import display
from datetime import date, datetime, time, timedelta
from dateutil.relativedelta import relativedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from skpy import Skype
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

import numpy as np
import openpyxl
import os
import pandas as pd
import pretty_errors
import time
import warnings
import winsound

pd.options.display.float_format = "{:.1f}".format
warnings.resetwarnings()
warnings.simplefilter("ignore")


dt = datetime.now()
dt_day, dt_month, dt_year, dt_week = dt.day, dt.month, dt.year, dt.weekday()

# 基準となる時間

# 現在時間
print((now_time := datetime.now().time()))
# 処理方法決定　<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

COMPANY_LOGIN = "87002"
COMPANY_PASS = "ZwP7&7sqhVDx"


# 定数としてファイル名を宣言
MONTHLY_TOTAL_FILE = '月次合計.pkl'
ABSENT_RECORD_FILE = '欠勤履歴.pkl'
PROJECT_RECORD_FILE = 'プロジェクト.pkl'
REGULATION_OVERTIME_FILE = '安衛法.pkl'
HOLIDAY_RECORD_FILE = '有休取得実績.pkl'
VACATION_RECORD_FILE = '休暇取得.pkl'

# 保存先フォルダの指定
SAVE_FOLDER = 'data'

##    Seleniumの起動

In [ ]:
def open_chrome(isHedless=True):
    options = webdriver.chrome.options.Options()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    options.use_chromium = True
    options.add_argument('--disable-extensions')
    # options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disk-cache-size=4096')

    if isHedless:
        options.add_argument("--headless")

    try:
        driver = webdriver.Chrome(
            ChromeDriverManager().install(),
            options=options)
    except BaseException:
        service = webdriver.chrome.service.Service()
        driver = webdriver.Chrome(service=service, options=options)
        print("service set")
    return driver

# 一般化ログイン
引数が4の場合は、labelの最初にボタンを押すタイプ

In [ ]:
def web_login(driver, URL, label, user, passwd):
    try:
        driver.get(URL)
        if len(label) == 4:
            driver.find_element(by="name", value=label[0]).click()
        # search_bar = driver.find_element(by='name', value=label[0])
        # search_bar.clear()
        # search_bar.send_keys(user)
        driver.find_element(by="name", value=label[-3]).send_keys(user)
        driver.find_element(by="name", value=label[-2]).send_keys(passwd)
        driver.find_element(by="name", value=label[-1]).click()
    except BaseException:
        pass
        print("")

In [ ]:
def schedule_approval(driver, offset=0):
    # offset = 1 if offset else 0
    URL = ("https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&@SUB=root.cws.shuro.boss.wp.approval&@"
           "SN=root.cws.shuro.boss.wp.approval&@FN=FORM_WP_BOSS&@ACTION_LOG_TXT=%E4%B8%8A%E9%95%B7%E7%94%A8%E5%8B%A4%E5%8B%99%E4%BA%88%E5%AE%9A%E6%89%BF%E8%AA%8D")
    try:
        driver.get(URL)
        search_bar = driver.find_element(by="name", value="@PTGTDTDAY")
        select = Select(search_bar)
        day = dt_day + offset
        select.select_by_value(str(day))
        driver.find_element(by="name", value="BTNLOADDAILY").click()
        driver.find_element(by="name", value="BTN_CHECK_ALL_LOWER").click()
        driver.find_element(by="name", value="BTNAPRV1").click()
        driver.find_element(by="name", value="BTNDCDE1").click()
    except BaseException:
        pass

In [ ]:
# 月次合計一覧

# Selenium 起動

In [ ]:
def get_from_month_type(pkl_name, to_year, to_month, default_year=2024, default_month=1):
    def SetMonth(driver,year,month):
        driver.find_element(by="name", value="doRetrievalButton" ).click()
        element =  driver.find_element(By.ID, "TCDR_NTYEAR")
        driver.execute_script(f"arguments[0].value = {year};", element)
        
        search0_bar = driver.find_element(By.NAME, "TCDR_NTMONTH")
        select = Select(search0_bar)
        select.select_by_index(int(month)-1)
        driver.find_element(by="id", value="OPT-COND@10" ).click()
    
    def GetMonth(driver):
        element =  driver.find_element(By.ID, "doRetrievalButton")

    def get_webdata(driver, XPASS):
        tableElem = driver.find_element(By.XPATH, value=XPASS)
        trs = tableElem.find_elements(By.TAG_NAME, "tr")
        data = []
        for tr in trs[1:]:
            data.append([td.text for td in tr.find_elements(By.TAG_NAME, "td")])
        columns_name = data[0]
        df_data = data[1:]
        df = pd.DataFrame(df_data, columns=columns_name)
        df = df.dropna(how='all')
        return df

    def read_pkl_data(pkl_name):
        try:
            df = pd.read_pickle(pkl_name)
            max_date = pd.to_datetime(df['年月'], errors='coerce').max()
            if max_date.day <= 4:
                adjusted_date = max_date - pd.DateOffset(months=1)
                return adjusted_date.year, adjusted_date.month, df
            else:
                return max_date.year, max_date.month, df
        except Exception as e:
            return None, None, pd.DataFrame()

    # URL1=r'https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&@SUB=root.cws.shuro.boss.aero_boss.aero_boss_menu006&@SN=root.cws.shuro.boss.aero_boss.aero_boss_menu006&@FN=form_aero_menu&@ACTION_LOG_TXT=%E6%9C%88%E6%AC%A1%E5%90%88%E8%A8%88%E4%B8%80%E8%A6%A7'
    XPASS = '/html/body/table[2]/tbody/tr/td/table/tbody/tr/td/form/table/tbody/tr[5]/td'  # 実際のXPathに置き換えてください

    from_year, from_month, df = read_pkl_data(pkl_name)

    if from_year is None or from_month is None:
        from_year, from_month = default_year, default_month
        df = pd.DataFrame() 
    
    # from_month が 12月で to_month が 翌年の 1月の場合などを考慮
    while (from_year < to_year) or (from_year == to_year and from_month <= to_month):
        SetMonth(driver, from_year, from_month)
        GetMonth(driver)
        web_data = get_webdata(driver, XPASS)
        # 1行目をカラム名として設定
        web_data['年月'] = f"{from_year}-{from_month}"
        df = pd.concat([df, web_data], axis=0, ignore_index=True)
        
        # 月を更新
        if from_month == 12:
            from_month = 1
            from_year += 1
        else:
            from_month += 1
    
    return df.drop_duplicates()

In [ ]:
def get_duration_type(pkl_name, to_year, to_month, from_year=2024, from_month=1):
    
    def SetMonth(driver, from_year, from_month, from_day, to_year, to_month):
        driver.find_element(by="name", value="OPT-COND").click()
        element = driver.find_element(By.ID, "TCDR_SDYEAR")
        driver.execute_script(f"arguments[0].value = {from_year};", element)
        element = driver.find_element(By.ID, "TCDR_EDYEAR")
        driver.execute_script(f"arguments[0].value = {to_year};", element)

        search0_bar = driver.find_element(By.NAME, "TCDR_SDMONTH")
        select = Select(search0_bar)
        select.select_by_index(int(from_month)-1)
        search0_bar = driver.find_element(By.NAME, "TCDR_EDMONTH")
        select = Select(search0_bar)
        select.select_by_index(int(to_month)-1)
        search0_bar = driver.find_element(By.NAME, "TCDR_SDDAY")
        select = Select(search0_bar)
        select.select_by_index(int(from_day)-1)
        search0_bar = driver.find_element(By.NAME, "TCDR_EDDAY")
        select = Select(search0_bar)
        select.select_by_index(int(get_last_day_of_month(to_year, to_month))-1)
        
        driver.find_element(By.NAME, value="doRetrievalButton").click()
    
    def get_last_day_of_month(year, month):
        import calendar
        # 月の最終日を取得
        last_day = calendar.monthrange(year, month)[1]
        return last_day

    def read_pkl_data(pkl_name):
        try:
            df = pd.read_pickle(pkl_name)
            if '日付' not in df.columns:
                print(f"ファイル {pkl_name} に「日付」カラムが見つかりませんでした。")
                return None, None, None

            three_days_before = pd.to_datetime(df['日付'], errors='coerce').max() - pd.Timedelta(days=3)
            return three_days_before.year, three_days_before.month, three_days_before.day
        
        except Exception as e:
            return None, None, None

    def get_webdata(driver, XPASS):
        tableElem = driver.find_element(By.XPATH, value=XPASS)
        trs = tableElem.find_elements(By.TAG_NAME, "tr")
        data = []
        for tr in trs[1:]:
            data.append([td.text for td in tr.find_elements(By.TAG_NAME, "td")])
        columns_name = data[0]
        df_data = data[1:]
        df = pd.DataFrame(df_data, columns=columns_name)
        df = df.dropna(how='all')
        return df

    XPASS = '/html/body/table[2]/tbody/tr/td/table/tbody/tr/td/form/table/tbody/tr[5]/td'  # 実際のXPathに置き換えてください
    
    from_year_pkl, from_month_pkl, from_day_pkl = read_pkl_data(pkl_name)
    
    # pklがなければデフォルト値を使用
    if from_year_pkl is None or from_month_pkl is None or from_day_pkl is None:
        from_year_pkl, from_month_pkl, from_day_pkl = from_year, from_month, 1

    SetMonth(driver, from_year_pkl, from_month_pkl, from_day_pkl, to_year, to_month)
    web_data = get_webdata(driver, XPASS)
    
    try:
        df = pd.read_pickle(pkl_name)
        df_combined = pd.concat([df, web_data], axis=0, ignore_index=True)
        return df_combined.drop_duplicates()
    except Exception as e:
        print(f"{e}")
        return web_data

# company へログイン

In [ ]:
try:
    driver.current_url
except BaseException:
    driver = open_chrome(False)

# company へログイン
label = ["uid", "pwd", "Login"]
URL = "https://delta-cws.company.works-hi.com/self-workflow/cws/cws"
web_login(driver, URL, label, COMPANY_LOGIN, COMPANY_PASS)

In [ ]:
%%time
# 変数の設定
today = date.today()
to_year = today.year
to_month = today.month

# 月次合計データの取得と保存
URL_hour = (
    r'https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&'
    r'@SUB=root.cws.shuro.boss.aero_boss.aero_boss_menu006&@SN=root.cws.shuro'
    r'.boss.aero_boss.aero_boss_menu006&@FN=form_aero_menu&@ACTION_LOG_TXT='
    r'%E6%9C%88%E6%AC%A1%E5%90%88%E8%A8%88%E4%B8%80%E8%A6%A7'
)
driver.get(URL_hour)
df_hours = get_from_month_type("月次合計.pkl", to_year, to_month)


# 欠勤履歴の取得と保存
URL_absent = (
    r'https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&'
    r'@SUB=root.cws.shuro.boss.aero_boss.aero_boss_menu008&@SN=root.cws.shuro'
    r'.boss.aero_boss.aero_boss_menu008&@FN=form_aero_menu&@ACTION_LOG_TXT='
    r'%E6%AC%A0%E5%8B%A4%E5%B1%A5%E6%AD%B4%E7%A2%BA%E8%AA%8D'
)
driver.get(URL_absent)
df_absent = get_duration_type("欠勤履歴.pkl", to_year, to_month)


# プロジェクト実績の取得と保存
URL_project = (
    r'https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&'
    r'@SUB=root.cws.shuro.boss.aero_boss.aero_boss_menu009&@SN=root.cws.shuro'
    r'.boss.aero_boss.aero_boss_menu009&@FN=form_aero_menu&@ACTION_LOG_TXT='
    r'%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E5%AE%9F%E7%B8%BE'
    r'%E9%9B%86%E8%A8%88%EF%BC%88%E7%A4%BE%E5%93%A1%E3%83%BB%E6%97%A5%E3%80%85'
    r'%EF%BC%89'
)
driver.get(URL_project)
df_wk = get_duration_type("プロジェクト.pkl", to_year, to_month)


# 安衛法超過時間管理の取得と保存
URL_regulation = (
    r'https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&'
    r'@SUB=root.cws.shuro.boss.aero_boss.aero_boss_menu013&@SN=root.cws.shuro'
    r'.boss.aero_boss.aero_boss_menu013&@FN=form_aero_menu&@ACTION_LOG_TXT='
    r'%E5%AE%89%E8%A1%9B%E6%B3%95%E3%80%80%E8%B6%85%E9%81%8E%E6%99%82%E9%96%93'
    r'%E7%AE%A1%E7%90%86'
)
driver.get(URL_regulation)
df_reg = get_from_month_type("安衛法.pkl", to_year, to_month)

# 有休取得実績の取得と保存
URL_holiday = (
    r'https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&'
    r'@SUB=root.cws.shuro.boss.aero_boss.aero_boss_menu004&@SN=root.cws.shuro'
    r'.boss.aero_boss.aero_boss_menu004&@FN=form_aero_menu&@ACTION_LOG_TXT='
    r'%E6%9C%89%E4%BC%91%E5%8F%96%E5%BE%97%E5%AE%9F%E7%B8%BE%E7%AE%A1%E7%90%86'
)
driver.get(URL_holiday)
df_holiday = get_from_month_type("有休取得実績.pkl", to_year, to_month)

# 休暇取得履歴の取得と保存
URL_vacation = (
    r'https://delta-cws.company.works-hi.com/self-workflow/cws/cws?@SID=null&'
    r'@SUB=root.cws.shuro.boss.aero_boss.aero_boss_menu007&@SN=root.cws.shuro'
    r'.boss.aero_boss.aero_boss_menu007&@FN=form_aero_menu&@ACTION_LOG_TXT='
    r'%E4%BC%91%E6%9A%87%E5%8F%96%E5%BE%97%E5%B1%A5%E6%AD%B4'
)
driver.get(URL_vacation)
df_vacation = get_duration_type("休暇取得.pkl", to_year, to_month)

# PICKLE書き込み

In [ ]:
# フォルダが存在しない場合は作成
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

# データの保存
df_hours.to_pickle(os.path.join(SAVE_FOLDER, MONTHLY_TOTAL_FILE))
df_absent.to_pickle(os.path.join(SAVE_FOLDER, ABSENT_RECORD_FILE))
df_wk.to_pickle(os.path.join(SAVE_FOLDER, PROJECT_RECORD_FILE))
df_reg.to_pickle(os.path.join(SAVE_FOLDER, REGULATION_OVERTIME_FILE))
df_holiday.to_pickle(os.path.join(SAVE_FOLDER, HOLIDAY_RECORD_FILE))
df_vacation.to_pickle(os.path.join(SAVE_FOLDER, VACATION_RECORD_FILE))

# pickleファイルからデータフレームを読み込む